In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Precinct Info

In [89]:
precints = pd.read_csv("/Users/mattiesanseverino/Downloads/precints.csv")

In [90]:
# Only use columns we need
precints = precints[['the_geom', 'Neigh22', 'Prec_2022']]
# Rename Prec_2022 to Precinct
precints = precints.rename(columns={'Prec_2022':'Precinct'})
for col in precints.columns:
    precints[col] = precints[col].apply(lambda x: f'"{x}"')

precints.head()

,the_geom,Neigh22,Precinct
0,"""MULTIPOLYGON (((-122.4216519998981 37.7102860...","""Visitacion Valley""","""7042"""
1,"""MULTIPOLYGON (((-122.45595499996385 37.711339...","""Outer Mission""","""1107"""
2,"""MULTIPOLYGON (((-122.44616799957213 37.711038...","""Excelsior""","""1145"""
3,"""MULTIPOLYGON (((-122.41508099966457 37.711655...","""Visitacion Valley""","""7043"""
4,"""MULTIPOLYGON (((-122.40972799997195 37.711951...","""Visitacion Valley""","""7046"""


In [91]:
# Reformat the_geom to be like lat lon, lat lon, lat lon
precints['the_geom'] = precints['the_geom'].apply(lambda x: ''.join(char for char in x if not char.isalpha() and not char == '(' and not char == ')'))
precints.head()

,the_geom,Neigh22,Precinct
0,""" -122.4216519998981 37.7102860001821, -122.42...","""Visitacion Valley""","""7042"""
1,""" -122.45595499996385 37.71133900031657, -122....","""Outer Mission""","""1107"""
2,""" -122.44616799957213 37.71103899979199, -122....","""Excelsior""","""1145"""
3,""" -122.41508099966457 37.71165500005792, -122....","""Visitacion Valley""","""7043"""
4,""" -122.40972799997195 37.71195199984119, -122....","""Visitacion Valley""","""7046"""


## Clean up Prop 33 Voting data

In [92]:
voting_info = pd.read_csv("/Users/mattiesanseverino/Downloads/voting_info_2.csv")

In [93]:
voting_info.head()

,Precinct,YES total,Yes percent,NO total,No percent,Total Votes,Unnamed: 6
0,PCT 1101,NaN,NaN,NaN,NaN,NaN,NaN
1,Election Day,48,50.53%,47,49.47%,95,NaN
2,Vote by Mail,204,39.61%,311,60.39%,515,NaN
3,Total,252,41.31%,358,58.69%,610,NaN
4,PCT 1102,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# Re structure the dataset

substrings_to_keep = 'PCT|Total'
filtered_df = voting_info[voting_info['Precinct'].str.contains(substrings_to_keep)]
filtered_df = filtered_df[['Precinct', 'Yes percent', 'No percent']]
filtered_df.reset_index(drop=True, inplace=True)
filtered_df.head()

,Precinct,Yes percent,No percent
0,PCT 1101,NaN,NaN
1,Total,41.31%,58.69%
2,PCT 1102,NaN,NaN
3,Total,41.12%,58.88%
4,PCT 1103 MB,NaN,NaN


In [95]:
filtered_df['Yes percent'] = filtered_df['Yes percent'].bfill()
filtered_df['No percent'] = filtered_df['No percent'].bfill()
filtered_df = filtered_df[filtered_df['Precinct'].str.contains('PCT')]

for col in filtered_df.columns:
    # stringify the cols
    filtered_df[col] = filtered_df[col].apply(lambda x: f'"{x}"')
    # make sure precinct is just the number
    filtered_df[col] = filtered_df[col].apply(lambda x: ''.join(char for char in x if not char.isalpha() and not char == ' '))

filtered_df.head()

,Precinct,Yes percent,No percent
0,"""1101""","""41.31%""","""58.69%"""
2,"""1102""","""41.12%""","""58.88%"""
4,"""1103""","""47.95%""","""52.05%"""
6,"""1104""","""39.86%""","""60.14%"""
8,"""1105""","""40.14%""","""59.86%"""


## Combine the tables

In [96]:
combined = pd.merge(filtered_df, precints, on='Precinct')

In [104]:
# Get rid of spaces
combined = combined.rename(columns={'Yes percent':'Yes', 'No percent': 'No'})
combined.head()

,Precinct,Yes,No,the_geom,Neigh22
0,"""1101""","""41.31%""","""58.69%""",""" -122.47167400018144 37.72163100026348, -122....","""Oceanview/Merced/Ingleside"""
1,"""1102""","""41.12%""","""58.88%""",""" -122.46801699998228 37.71610300017965, -122....","""Oceanview/Merced/Ingleside"""
2,"""1103""","""47.95%""","""52.05%""",""" -122.46625399964336 37.719679000222065, -122...","""Oceanview/Merced/Ingleside"""
3,"""1104""","""39.86%""","""60.14%""",""" -122.46266299956643 37.72000400008428, -122....","""Oceanview/Merced/Ingleside"""
4,"""1105""","""40.14%""","""59.86%""",""" -122.46263300032751 37.71742100005925, -122....","""Oceanview/Merced/Ingleside"""


## Export the data

In [105]:
combined.to_csv('prop_33_results.csv')